In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Uncertainty analysis for XGB(ADF)
---------------------------------
- Reads all HHO-optimized configurations (typically 100 rows) from HHO_XGB_results_SSSI.xlsx
- For each config: split data 70/30 with the same iteration-based random_state, fit XGB,
  and collect predictions paired with their *true* thickness values (train + test).
- For each unique thickness value, compute mean, 2.5th and 97.5th percentiles -> 95% CI.
- Exports a small figure of the mean curve with shaded 95% CI.

This matches the manuscript’s uncertainty idea and keeps the core training choices consistent.
"""

import os
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- paths ---
PARAM_XLSX = "HHO_XGB_results_SSSI.xlsx"   # results with 100 rows for SSSI group
TRAIN_XLSX = "Train_data.xlsx"             # training data with 7 features + 'thickness'
OUT_FIG    = "uncertainty_ci.png"

# --- features/target ---
FEATURES = ['SSSI', 'SSP', 'DEM', 'MRRTF', 'NDVI', 'MRVBF', 'RD']
TARGET   = 'thickness'

# --- load data ---
df_params = pd.read_excel(PARAM_XLSX)
df_data   = pd.read_excel(TRAIN_XLSX).dropna()

# filter to SSSI group if column exists
if "Group" in df_params.columns:
    df_params = df_params[df_params["Group"] == "SSSI"].copy()

# sanity checks
need_cols = set(FEATURES + [TARGET])
missing = [c for c in need_cols if c not in df_data.columns]
if missing:
    raise ValueError(f"Missing columns in training data: {missing}")
if df_params.empty:
    raise ValueError("No parameter rows found for SSSI in the results file.")

X_all = df_data[FEATURES].values
y_all = df_data[TARGET].values

# map: true thickness -> list of predicted values across all runs
unique_thk = np.unique(y_all).astype(float)
pred_bag = {float(t): [] for t in unique_thk}

# iterate all HHO configurations (typically 100)
for idx, row in tqdm(df_params.iterrows(), total=len(df_params), desc="Uncertainty runs"):
    # seed: use iteration directly if present; else fall back to row index + 1
    if "Iteration" in row and not pd.isna(row["Iteration"]):
        seed0 = int(row["Iteration"])
    else:
        seed0 = int(idx) + 1

    model = XGBRegressor(
        max_depth=int(row['max_depth']),
        learning_rate=float(row['learning_rate']),
        n_estimators=int(row['n_estimators']),
        objective="reg:squarederror",
        random_state=seed0,
        verbosity=0
    )

    # use 70/30 split to match the manuscript’s repeated training setup
    X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, train_size=0.7, random_state=seed0)
    model.fit(X_tr, y_tr)

    # collect predictions with their ground-truth thickness
    yhat_tr = model.predict(X_tr)
    yhat_te = model.predict(X_te)
    for t_true, yhat in zip(y_tr, yhat_tr):
        pred_bag[float(t_true)].append(float(yhat))
    for t_true, yhat in zip(y_te, yhat_te):
        pred_bag[float(t_true)].append(float(yhat))

# aggregate to mean and 95% CI per thickness
thk_list, mean_list, lo_list, hi_list = [], [], [], []
for t in sorted(pred_bag.keys()):
    preds = np.array(pred_bag[t], dtype=float)
    if preds.size == 0:
        continue
    thk_list.append(t)
    mean_list.append(preds.mean())
    lo_list.append(np.percentile(preds, 2.5))
    hi_list.append(np.percentile(preds, 97.5))

thicknesses = np.array(thk_list)
means       = np.array(mean_list)
los         = np.array(lo_list)
his         = np.array(hi_list)

# plot
plt.figure(figsize=(3.67, 2.52))
plt.fill_between(thicknesses, los, his, alpha=0.3, label="95% Confidence Interval")
plt.plot(thicknesses, means, linewidth=1, label="Mean Prediction")

plt.xlabel("Soil Thickness (m)", fontsize=9)
plt.ylabel("Prediction (m)", fontsize=9)
plt.grid(True, linewidth=0.4, alpha=0.4)
plt.legend(frameon=False, loc="upper left")
plt.tight_layout()
plt.savefig(OUT_FIG, dpi=600, bbox_inches="tight")
# plt.show()

print(f"[DONE] Uncertainty curve saved -> {OUT_FIG}")
